In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [30]:
df = pd.read_csv('Churn_Modelling.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [31]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [32]:
df.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [33]:
#preprocessing

df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [34]:
label_gender = LabelEncoder()

df['Gender'] = label_gender.fit_transform(df['Gender'])

In [35]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [36]:
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder()
encoder = ohe_geo.fit_transform(df[['Geography']])
encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [37]:
ohe_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [38]:
goe_encoded_df = pd.DataFrame(encoder.toarray(), columns=ohe_geo.get_feature_names_out(['Geography']))

In [39]:
goe_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [40]:
df.drop(columns=['Geography'], axis=1, inplace=True)
df = pd.concat([df, goe_encoded_df], axis=1)

In [41]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [42]:
#pickling encoders
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)
with open('label_encoder_gender.pkl', 'wb') as genderfile:
    pickle.dump(label_gender, genderfile)

In [43]:
#dividing dataset
y = df['Exited']
X = df.drop(columns=['Exited'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scaling features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [44]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Creating ANN

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

## Model description

In [46]:
model = Sequential(
    [
        Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)),# HL1
        Dense(32, activation = 'relu'), #HL2
        Dense(1, activation = 'sigmoid') #OPL
    ]
)

In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
import tensorflow
opt = tensorflow.keras.optimizers.AdamW(learning_rate = 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [49]:
## Compile model
model.compile(optimizer = opt, loss = loss, metrics=['accuracy'])

### Setup the TensorBorad

In [50]:
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)

In [51]:
# Set up Early Stopping

early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)

In [52]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
    )

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4015 - accuracy: 0.8301 - val_loss: 0.3503 - val_accuracy: 0.8555
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3581 - accuracy: 0.8516 - val_loss: 0.3478 - val_accuracy: 0.8520
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3483 - accuracy: 0.8585 - val_loss: 0.3486 - val_accuracy: 0.8590
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3464 - accuracy: 0.8587 - val_loss: 0.3574 - val_accuracy: 0.8565
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3414 - accuracy: 0.8615 - val_loss: 0.3414 - val_accuracy: 0.8585
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3400 - accuracy: 0.8616 - val_loss: 0.3494 - val_accuracy: 0.8530
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3381 - accuracy: 0.8601 - val_loss: 0.3507 - val_accuracy: 0.8550

In [53]:
model.save('model.h5')

c:\Users\HP\Desktop\ML_Course_Datasets\ANN_project\annmodel\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [54]:
# load Tensorflow board

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [59]:
%tensorboard --logdir logs/fit --host localhost

Reusing TensorBoard on port 6006 (pid 25392), started 0:02:02 ago. (Use '!kill 25392' to kill it.)